In [1]:
import MeCab
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import pymysql
import sqlalchemy
import re
import ast
from gensim.models import Word2Vec
import math

In [3]:
conn = pymysql.connect(host='127.0.0.1', user='root', password='1234',db='esg_db', charset='utf8mb4')
cursor = conn.cursor()


In [2]:
    df = pd.read_csv('esg_dataset_2021_cleansing.csv')
    df['noun']  = df['noun'].map(lambda x : ast.literal_eval(x))
    df.head(5)

,title,url,press,date,content,noun
0,"올해는 ESG 정보공시 표준화 원년, 압박 더 거세진다",https://news.naver.com/main/read.naver?mode=LS...,머니투데이,2021.01.01.,() 산업·금융·법조계 ESG 전문가 대담]/사진제공=이미지투데이2020년은 E...,"[산업, 금융, 법조, ESG, 전문가, 대담, 사진, 제공, 이미지, ESG, 환..."
1,"서학개미 사로잡은 美 ESG ETF, 올해도 高高할까",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,() 작년 착한 투자에 30조 몰려인베스코솔라 年 급등美기업 IPO 열기도 뜨거...,"[작년, 투자, 려인, 베스코, 솔라, 年, 급등, 美, 기업, IPO, 로빈, 후..."
2,"[신년사]방문규 수은 행장 ""수출 6000억달러 선봉장될 것""",https://news.naver.com/main/read.naver?mode=LS...,뉴시스,2021.01.01.,() ESG 경영 강화...수출 중소중견기업 맞춤형 지원 방문규 수출입은행장 (사...,"[ESG, 경영, 강화, 수출, 중소, 중견, 기업, 맞춤, 지원, 방문, 규, 수..."
3,[韓산업 대전환 ①] 격변기의 시대…대전환 앞당겼다,https://news.naver.com/main/read.naver?mode=LS...,아이뉴스24,2021.01.01.,() 코로나 팬데믹에 뉴노멀 시대 활짝…대내외 경제 불안 속 미래 먹거리 찾기 총...,"[코로나, 믹, 뉴노, 멀, 시대, 대내외, 경제, 불안, 속, 미래, 먹거리, 총..."
4,"펀드사태 혼쭐난 금융권, 외부 `포청천` 영입",https://news.naver.com/main/read.naver?mode=LS...,매일경제,2021.01.01.,"() 4대 금융지주 인사 키워드는우리금융, 감사원 출신 신민철하나銀은 이인영 변호...","[금융, 지주, 인사, 키워드, 금융, 감사원, 출신, 신민철, 銀, 이인영, 변호..."


In [3]:
    tokenized_data = []
    for noun_list in df['noun']:
        tokenized_data.append(noun_list)
    tokenized_data[:3]

[['산업',
  '금융',
  '법조',
  'ESG',
  '전문가',
  '대담',
  '사진',
  '제공',
  '이미지',
  'ESG',
  '환경',
  '사회',
  '가치',
  '지배',
  '구조',
  '논의',
  '원년',
  '평가',
  '포털',
  '사이트',
  '다음',
  'Daum',
  'ESG',
  '키워드',
  '검색',
  '건수',
  '기준',
  '간',
  '전체',
  'ESG',
  '건수',
  '배',
  'ESG',
  '주목',
  '지난해',
  '전',
  '세계',
  '코로나',
  'COVID',
  '전염병',
  '경제',
  '시스템',
  '전반',
  '충격',
  '정부',
  '기업',
  '그간',
  '재무',
  '리스크',
  '요인',
  '관리',
  '필요',
  '절감',
  '주요',
  '기업',
  '금융',
  '사',
  'ESG',
  '전사',
  '리스크',
  '관리',
  '전략',
  '기획',
  '차원',
  '시작',
  '올해',
  '축년',
  '기업',
  '금융',
  '법조',
  '다양',
  '분야',
  '활동',
  'ESG',
  '전문가',
  '초빙',
  'ESG',
  '규제',
  '환경',
  '변화',
  '기업',
  '금융',
  '길',
  '주제',
  '대담',
  '진행',
  '이번',
  '대담',
  '권기준',
  'SK',
  '수',
  '펙',
  '협의회',
  'SV',
  '위원회',
  '부사장',
  '김성우',
  '김',
  '애',
  '장법',
  '사무소',
  '환경',
  '에너지',
  '연구',
  '소장',
  '우상현',
  'KB',
  '은행',
  'CIB',
  '기업',
  '투자',
  '금융',
  '전무',
  'KB',
  '증권',
  'IB',
  '투자',
  '은행',
  '부사장',
  '윤석모',
  '삼성증권',
  '리

In [5]:
count_table = pd.DataFrame(columns = ['noun','count','total','probability'])
pmi_table = pd.DataFrame(columns = ['noun1','noun2','co-occurence','pmi'])

In [ ]:
corp = tokenized_data
list_pmi = []

dic={}
tot=0

dic2d={}
for d in tqdm(corp):
    for w in d:
        tot+=1
        if w not in dic:
            dic[w]=1
        else:
            dic[w]+=1

    l=len(d)
    for i1 in range(0,l-1):
        for i2 in range(i1+1, l):
            w1=d[i1]
            w2=d[i2]
            if w1 not in dic2d:
                dic2d[w1]={}
            if w2 not in dic2d[w1]:
                dic2d[w1][w2]=1
            else:
                dic2d[w1][w2]+=1

            if w2 not in dic2d:
                dic2d[w2]={}
            if w1 not in dic2d[w2]:
                dic2d[w2][w1]=1
            else:
                dic2d[w2][w1]+=1

print(tot)
                
for key in tqdm(dic.keys()):
    pr = dic[key]/tot
    #print(key, dic[key], tot, pr)
    aa = {'noun1' : key, 
          'count' : dic[key],
          'total' : tot,
          'probability' : pr}
    #count_table.append(aa)
    #print('-'*50)


pmi={}
for key1 in tqdm(dic2d.keys()):
    dsub=dic2d[key1]
    pmi[key1]={}
    for key2 in dsub.keys():
        pmiValue=math.log((dic2d[key1][key2]/tot)/((dic[key1]/tot)*(dic[key2]/tot))) #pmi 수식(값이 높을수록 관련성이 높다.)
        #print(key1, key2, dic2d[key1][key2], pmiValue)
        bb = {'noun1' : key1, 
              'noun2' : key2,
              'co-occurence' : dic2d[key1][key2],
              'pmi' : pmiValue}
        list_pmi.append(bb)
        pmi[key1][key2]=pmiValue

  0%|          | 0/3472 [00:00<?, ?it/s]

956512


  0%|          | 0/30284 [00:00<?, ?it/s]

  0%|          | 0/30283 [00:00<?, ?it/s]

In [6]:
# 단순 카운트
dic

{'산업': 2227,
 '금융': 5809,
 '법조': 25,
 'ESG': 27176,
 '전문가': 803,
 '대담': 20,
 '제공': 2384,
 '이미지': 232,
 '환경': 8074,
 '사회': 10465,
 '가치': 2758,
 '지배': 3827,
 '구조': 4211,
 '논의': 539,
 '원년': 126,
 '평가': 4255,
 '포털': 55,
 '사이트': 76,
 '다음': 357,
 'Daum': 1,
 '키워드': 166,
 '검색': 46,
 '건수': 37,
 '기준': 1911,
 '간': 600,
 '전체': 670,
 '배': 494,
 '주목': 479,
 '지난해': 2222,
 '전': 807,
 '세계': 2036,
 '코로나': 1813,
 'COVID': 26,
 '전염병': 15,
 '경제': 2216,
 '시스템': 1119,
 '전반': 491,
 '충격': 43,
 '정부': 1287,
 '기업': 14493,
 '그간': 102,
 '재무': 940,
 '리스크': 801,
 '요인': 236,
 '관리': 1863,
 '필요': 1426,
 '절감': 461,
 '주요': 1369,
 '사': 1397,
 '전사': 251,
 '전략': 2591,
 '기획': 438,
 '차원': 509,
 '시작': 1034,
 '올해': 2436,
 '축년': 11,
 '다양': 933,
 '분야': 2151,
 '활동': 3078,
 '초빙': 19,
 '규제': 628,
 '변화': 2023,
 '길': 154,
 '주제': 416,
 '진행': 1909,
 '이번': 2142,
 '권기준': 4,
 'SK': 3257,
 '수': 423,
 '펙': 44,
 '협의회': 293,
 'SV': 66,
 '위원회': 3502,
 '부사장': 296,
 '김성우': 2,
 '김': 508,
 '애': 298,
 '장법': 2,
 '사무소': 46,
 '에너지': 3059,
 '연구': 849,
 

In [20]:
print(list_pmi[100]['noun1'])

산업


In [29]:
article_esg_pmi_result = []
for i in range(len(list_pmi)) :
    if list_pmi[i]['noun1'] == 'ESG' :
        article_esg_pmi_result.append(list_pmi[i])
print(article_esg_pmi_result)

[{'noun1': 'ESG', 'noun2': '산업', 'co-occurence': 23574, 'pmi': 5.918981307764587}, {'noun1': 'ESG', 'noun2': '금융', 'co-occurence': 81505, 'pmi': 6.200748215283298}, {'noun1': 'ESG', 'noun2': '법조', 'co-occurence': 473, 'pmi': 6.499711849859172}, {'noun1': 'ESG', 'noun2': '전문가', 'co-occurence': 12542, 'pmi': 6.307975863391644}, {'noun1': 'ESG', 'noun2': '대담', 'co-occurence': 716, 'pmi': 7.137440179642095}, {'noun1': 'ESG', 'noun2': '제공', 'co-occurence': 21458, 'pmi': 5.756810073246891}, {'noun1': 'ESG', 'noun2': '이미지', 'co-occurence': 3271, 'pmi': 6.205605942107514}, {'noun1': 'ESG', 'noun2': 'ESG', 'co-occurence': 492868, 'pmi': 6.457399445090122}, {'noun1': 'ESG', 'noun2': '환경', 'co-occurence': 83753, 'pmi': 5.898715254763836}, {'noun1': 'ESG', 'noun2': '사회', 'co-occurence': 119819, 'pmi': 5.997438201106317}, {'noun1': 'ESG', 'noun2': '가치', 'co-occurence': 31970, 'pmi': 6.00978446993608}, {'noun1': 'ESG', 'noun2': '지배', 'co-occurence': 41920, 'pmi': 5.953174119804662}, {'noun1': 'ESG',

In [40]:
article_esg_pmi_data = pd.DataFrame(article_esg_pmi_result)
article_esg_pmi_data

,noun1,noun2,co-occurence,pmi
0,ESG,산업,23574,5.918981
1,ESG,금융,81505,6.200748
2,ESG,법조,473,6.499712
3,ESG,전문가,12542,6.307976
4,ESG,대담,716,7.137440
...,...,...,...,...
29873,ESG,적어,1,3.559492
29874,ESG,청량감,1,3.559492
29875,ESG,코소브,1,3.559492
29876,ESG,Ecosobre,1,3.559492


In [41]:
article_esg_pmi_data_sort = article_esg_pmi_data.sort_values(by=['pmi'], ascending=[False])
article_esg_pmi_data_sort = article_esg_pmi_data_sort.reset_index()

article_esg_pmi_data_sort.pop("index")
article_esg_pmi_data_sort

,noun1,noun2,co-occurence,pmi
0,ESG,하인스,242,8.355283
1,ESG,도미니,363,8.355283
2,ESG,세비,121,8.355283
3,ESG,오피,121,8.355283
4,ESG,머스,121,8.355283
...,...,...,...,...
29873,ESG,초음파,2,2.643202
29874,ESG,캠벨,3,2.578663
29875,ESG,다이아몬드,1,2.460880
29876,ESG,스리,2,2.460880


In [45]:
article_esg_pmi_data_sort.to_csv('article_esg_pmi_data.csv', index=False, encoding="utf-8-sig")

In [46]:
article_esg_pmi_data_sort[:50]

,noun1,noun2,co-occurence,pmi
0,ESG,하인스,242,8.355283
1,ESG,도미니,363,8.355283
2,ESG,세비,121,8.355283
3,ESG,오피,121,8.355283
4,ESG,머스,121,8.355283
5,ESG,RTS,121,8.355283
6,ESG,VVIP,121,8.355283
7,ESG,송종호,121,8.355283
8,ESG,이광렬,121,8.355283
9,ESG,나자,121,8.355283
